In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np

TensorFlow 2.x selected.


In [2]:
print(tf.__version__)

2.0.0


# Upload your dataset

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [11]:
!unzip -x "/content/gdrive/My Drive/2.project/남동발전(안전장비검출 프로젝트)/2.데이터셋관리/person_with_out_helmet.zip"

Archive:  /content/gdrive/My Drive/2.project/남동발전(안전장비검출 프로젝트)/2.데이터셋관리/person_with_out_helmet.zip
   creating: person_with_out_helmet/
   creating: person_with_out_helmet/with_helmet/
  inflating: person_with_out_helmet/with_helmet/00062.jpg  
  inflating: person_with_out_helmet/with_helmet/00057.jpg  
  inflating: person_with_out_helmet/with_helmet/00050.jpg  
  inflating: person_with_out_helmet/with_helmet/00026.jpg  
  inflating: person_with_out_helmet/with_helmet/00022.jpg  
  inflating: person_with_out_helmet/with_helmet/00021.jpg  
  inflating: person_with_out_helmet/with_helmet/00011.jpg  
  inflating: person_with_out_helmet/with_helmet/00010.jpg  
  inflating: person_with_out_helmet/with_helmet/00005.jpg  
  inflating: person_with_out_helmet/with_helmet/00003.jpg  
  inflating: person_with_out_helmet/with_helmet/01838.jpg  
  inflating: person_with_out_helmet/with_helmet/01818.jpg  
  inflating: person_with_out_helmet/with_helmet/01813.jpg  
  i

In [12]:
data_dir = '/content/person_with_out_helmet'
print(data_dir)

/content/person_with_out_helmet


# Data pipeline

In [0]:
[IMG_WIDTH, IMG_HEIGHT] = [128, 128]

In [0]:
import os

In [15]:
os.listdir(data_dir)

['without_helmet', 'with_helmet']

In [16]:
path_list = []
extension_check = []
for label in os.listdir(data_dir):
  print(label)
  each_class_image_path = os.path.join(data_dir, label)
  
  for file in os.listdir(each_class_image_path):
    if file.split('.')[1] not in extension_check:
      extension_check.append(file.split('.')[1])
    path_list.append( os.path.join(each_class_image_path, file))

print('Kinds of extension : ', extension_check)
print(path_list[:10])
print('The number of data : ', len(path_list))

without_helmet
with_helmet
Kinds of extension :  ['bmp', 'jpg']
['/content/person_with_out_helmet/without_helmet/person_453.bmp', '/content/person_with_out_helmet/without_helmet/person_199.bmp', '/content/person_with_out_helmet/without_helmet/person_042.bmp', '/content/person_with_out_helmet/without_helmet/person_038.bmp', '/content/person_with_out_helmet/without_helmet/person_459.bmp', '/content/person_with_out_helmet/without_helmet/person_184.bmp', '/content/person_with_out_helmet/without_helmet/person_358.bmp', '/content/person_with_out_helmet/without_helmet/person_291.bmp', '/content/person_with_out_helmet/without_helmet/person_447.bmp', '/content/person_with_out_helmet/without_helmet/person_031.bmp']
The number of data :  3634


In [0]:
dataset = tf.data.Dataset.from_tensor_slices(path_list)

In [0]:
dataset = dataset.shuffle(3634).batch(64).repeat()

In [0]:
def path_process(path):
  parts = tf.strings.split(path, '/')
  label = parts[-1] # The second to last is the class-directory
  return label

def decode_img(path):
  img = tf.io.read_file(path)  
  if tf.strings.split(path, '.')[-1] == 'jpg':
    img = tf.image.decode_jpeg(img, channels=3)
  else:
    img = tf.image.decode_bmp(img, channels=3)
  
  img = tf.image.convert_image_dtype(img, tf.float32)
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])  

def data_process(path):
  [filepath, label] = path_process(path)
  #print(filepath.numpy())
  img = tf.io.read_file(filepath)
  #img = decode_img(filepath)
  return img, label

In [117]:
path_process('/content/person_with_out_helmet/without_helmet/person_453.bmp')

<tf.Tensor: id=3006, shape=(), dtype=string, numpy=b'person_453.bmp'>

In [0]:
path_list = ['/content/person_with_out_helmet/without_helmet/person_453.bmp',
            '/content/person_with_out_helmet/without_helmet/person_503.bmp']


In [0]:
new_dataset = tf.data.Dataset.from_tensor_slices(path_list)
new_dataset = dataset.shuffle(1).batch(1).repeat()

In [0]:
new_dataset = new_dataset.map(path_process)

In [122]:
next(iter(new_dataset)).shape

TensorShape([1, None])

In [113]:
for i in new_dataset.take(1):
  print(i)

tf.Tensor(b'person_453.bmp', shape=(), dtype=string)


In [0]:
img = tf.io.read_file('/content/person_with_out_helmet/without_helmet/person_453.bmp')

In [0]:
img = tf.image.decode_bmp(img, channels=3)

In [89]:
img_dataset = dataset.map(data_process)

ValueError: ignored